# 5.	Typos (6 points) 

In [3]:
# !pip install --user checklist
# !pip install --user tabulate

In [9]:
import numpy as np
import pandas as pd
import os
import logging
import checklist
from checklist.perturb import Perturb

# reseed
np.random.seed(42)

# text
cur_dir = os.getcwd()
subset_test = pd.read_csv(cur_dir + '/input/olid-data/olid-subset-diagnostic-tests.csv')
text = subset_test['text'].tolist()

In [10]:
# overload existing method from checklist to add more than 1 typo
def add_typos(string, typos=5):
        """Perturbation functions, swaps random characters with their neighbors
        Parameters
        ----------
        string : str
            input string
        typos : int
            number of typos to add
        Returns
        -------
        list(string)
            perturbed strings
        """
        string = list(string)
        swaps = np.random.choice(len(string) - 1, typos)
        for swap in swaps:
            tmp = string[swap]
            string[swap] = string[swap + 1]
            string[swap + 1] = tmp
        return ''.join(string)

In [11]:
# add_typos
typos = Perturb.perturb(text, add_typos)
#print(typos.data)

# plug back into pandas|
new_text = []
for t in typos.data:
    new_text.append(t[1])
subset_test['text_typos'] = new_text
print(subset_test)

       id                                               text  labels  \
0   89200      @USER @USER Who the hell does he think he is?       1   
1   71294  #BREAKING. #Greece: Molotov cocktails fly afte...       1   
2   55633  #OrrinHatch I can’t believe this sexist , clue...       1   
3   16856  @USER @USER I'll use that one the next time im...       1   
4   26072    0-1 lost my acca on the first fucking fight cba       1   
..    ...                                                ...     ...   
95  45518  @USER He is obviously getting suspended. He is...       0   
96  51610  #Canada - EXCLUSIVE: #Trudeau #Liberals leave ...       0   
97  26758  @USER @USER ...than why did you show us how ho...       0   
98  30718  @USER @USER @USER You have yet to answer what ...       0   
99  82558  #MAGA  ... got any ideas how she could have do...       0   

                                           text_typos  
0       @USER @SUER Wh othe hell doe she thinkh e si?  
1   #BREAKING. #Greece:

In [12]:
# Preparing test data
subset_test_df = subset_test[['text','labels']]
subset_test_list = subset_test_df['text'].values.tolist()

# Preparing typo test data
subset_typos_test_df = subset_test[['text_typos','labels']]
subset_typos_test_list = subset_typos_test_df['text_typos'].values.tolist()

# using model trained previously
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Make predictions with the model
subset_predictions, subset_raw_outputs = model.predict(subset_test_list)
#print(predictions, raw_outputs)

# Make predictions with the model on typos dataset
subset_typos_predictions, subset_typos_raw_outputs = model.predict(subset_typos_test_list)
#print(predictions, raw_outputs)

# Attach predictions to test df for evaluation
subset_test_df['predictions'] = subset_predictions

# Attach predictions to typo test df for evaluation
subset_typos_test_df['predictions'] = subset_typos_predictions

# get class distributions for getting weighted F1 score later
subset_freq_0 = subset_test_df['labels'].value_counts(normalize=True).iloc[0]
subset_freq_1 = subset_test_df['labels'].value_counts(normalize=True).iloc[1]
# note that class distributions are equal in this subset so weighted and macro
# F1 scores will be the same

# Calculate evaluation metrics
evaluation(subset_test_df, subset_freq_0, subset_freq_1)
print('Correctly identified messages in data: ' + str(sum(subset_test_df['predictions'] == subset_test_df['labels'])))
print('Total number of messages: ' + str(len(subset_test_df['predictions'])))

# Calculate evaluation metrics for typos
evaluation(subset_typos_test_df, subset_freq_0, subset_freq_1)
print('Correctly identified messages in typo data: ' + str(sum(subset_typos_test_df['predictions'] == subset_typos_test_df['labels'])))
print('Total number of messages: ' + str(len(subset_typos_test_df['predictions'])))

NameError: name 'model' is not defined

In [34]:
# save dataframes to html
subset_test_df.to_html('/kaggle/working/subset_test_df.html')
subset_typos_test_df.to_html('/kaggle/working/subset_typos_test_df.html')

# read dataframes from html
#subset_test_df = pd.read_html('../input/olid-subset-results/subset_test_df.html', index_col=0)[0]
#subset_typos_test_df = pd.read_html('../input/olid-subset-results/subset_typos_test_df.html', index_col=0)[0]

In [35]:
print(subset_typos_test_df['text_typos'][4])
print(subset_typos_test_df['text_typos'][16])
print(subset_typos_test_df['text_typos'][29])

In [36]:
pip install spacy

In [37]:
import spacy
nlp = spacy.load('en_core_web_sm')
pdata = list(nlp.pipe(text))

#print(pdata)
# reload data
subset_test = pd.read_csv('../input/olid-data/olid-subset-diagnostic-tests.csv')
text = subset_test['text'].tolist()

In [38]:
# add_negations
negations = Perturb.perturb(pdata, Perturb.add_negation)
#print(negations.data)
#print(len(subset_test))
#print(len(negations.data))

ids = []
text = []
text_negations = []
labels = []

# build negations dataframe from scratch since the method uses spacy
for n in negations.data:
    for index, row in subset_test.iterrows():
        if n[0] == row['text']:
            ids.append(row['id'])
            text.append(n[0])            
            labels.append(row['labels'])
            text_negations.append(n[1])

# initialize data of lists.
neg_data = {'id': ids, 'text': text,  'labels': labels, 'text_negations': text_negations}
subset_test = pd.DataFrame(neg_data)
print(subset_test)

In [39]:
# Preparing test data
subset_test_df = subset_test[['text','labels']]
subset_test_list = subset_test_df['text'].values.tolist()

# Preparing negations test data
subset_negs_test_df = subset_test[['text_negations','labels']]
subset_negs_test_list = subset_negs_test_df['text_negations'].values.tolist()

# using model trained previously

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Make predictions with the model
subset_predictions, subset_raw_outputs = model.predict(subset_test_list)
#print(predictions, raw_outputs)

# Make predictions with the model on negations dataset
subset_negs_predictions, subset_negs_raw_outputs = model.predict(subset_negs_test_list)
#print(predictions, raw_outputs)

# Attach predictions to test df for evaluation
subset_test_df['predictions'] = subset_predictions

# Attach predictions to negations test df for evaluation
subset_negs_test_df['predictions'] = subset_negs_predictions

# get class distributions for getting weighted F1 score later
subset_freq_0 = subset_test_df['labels'].value_counts(normalize=True).iloc[0]
subset_freq_1 = subset_test_df['labels'].value_counts(normalize=True).iloc[1]
# note that class distributions are equal in this subset so weighted and macro
# F1 scores will be the same

# Calculate evaluation metrics
evaluation(subset_test_df, subset_freq_0, subset_freq_1)
print('Correctly identified messages in data: ' + str(sum(subset_test_df['predictions'] == subset_test_df['labels'])))
print('Total number of messages: ' + str(len(subset_test_df['predictions'])))

# Calculate evaluation metrics for negations
evaluation(subset_negs_test_df, subset_freq_0, subset_freq_1)
print('Correctly identified messages in negations data: ' + str(sum(subset_negs_test_df['predictions'] == subset_negs_test_df['labels'])))
print('Total number of messages: ' + str(len(subset_negs_test_df['predictions'])))

In [40]:
# save dataframes to html
#subset_test_df.to_html('/kaggle/working/subset_test_df.html')
subset_negs_test_df.to_html('/kaggle/working/subset_negs_test_df.html')

# read dataframes from html
#subset_test_df = pd.read_html('../input/olid-subset-results/subset_test_df.html', index_col=0)[0]
#subset_typos_test_df = pd.read_html('../input/olid-subset-results/subset_typos_test_df.html', index_col=0)[0]

In [41]:
# question 7
from checklist.editor import Editor

editor = Editor()

In [60]:
# getting masked language model suggestions
hate = editor.suggest('I hate {mask}.')[:30]
not_hate = editor.suggest('I don\'t hate {mask}.')[:30]

hate_mask = []
for i in hate:
    hate_mask.append('I hate ' + i)
#print(hate_mask)

not_hate_mask = []
for i in hate:
    not_hate_mask.append('I don\'t hate ' + i)
#print(not_hate_mask)

# using the built-in lexicon, we also explore more specific mask subjects
ret1 = editor.template('I hate {nationality}.')
ret2 = editor.template('I don\'t hate {nationality}.')
ret3 = editor.template('I hate {religion}.')
ret4 = editor.template('I don\'t hate {religion}.')

hate_nationalities = list(np.random.choice(ret1.data, 10))
nationalities = list(np.random.choice(ret2.data, 10))
hate_religions = list(np.random.choice(ret3.data, 10))
religions = list(np.random.choice(ret4.data, 10))

# build dataset
hate_list = hate_mask + not_hate_mask + hate_nationalities + nationalities + hate_religions + religions
hate_df = pd.DataFrame({'text': hate_list})
print(hate_df)

In [57]:
# Make predictions with the model on hate dataset
hate_predictions, hate_raw_outputs = model.predict(hate_list)

# Attach predictions to test df for evaluation
hate_df['predictions'] = hate_predictions

print(hate_df)

In [59]:
# save hate result to html
hate_df.to_html('/kaggle/working/hate_df.html')